# 08.베스트 모델
### 최고의 모델을 찾기 위한 다양한 방법을 살펴보겠습니다.

# 와인 종류 예측: 데이터 확인
### 포르투갈 서북쪽 비뉴 베르드(Vinho Verde) 지방의 와인을 측정한 데이터
* 레드 와인 1,599개와 화이트 와인 4,898개를 등급과 맛, 산도를 측정하고 분석한 데이터
* dataset/wine.csv

### 데이터 설명
* 샘플 수: 6497
* 속성 수: 12
    - 컬럼 1: 주석산 농도
    - 컬럼 2: 아세트산 농도
    - 컬럼 3: 구연산 농도
    - 컬럼 4: 잔류 당분 농도
    - 컬럼 5: 염화나트륨 농도
    - 컬럼 6: 유리 아황산 농도
    - 컬럼 7: 총 아황산 농도
    - 컬럼 8: 밀도
    - 컬럼 9: pH
    - 컬럼 10: 황산칼륨 농도
    - 컬럼 11: 알코올 도수
    - 컬럼 12: 와인의 맛(0~10등급)
* 클래스 : 레드 와인(1), 화이트와인(0)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

import pandas as pd
import numpy as np
import tensorflow as tf

# seed 값 설정
# seed = 0
# np.random.seed(seed)
# tf.set_random_seed(seed)

# 데이터 입력
df_pre = pd.read_csv('./dataset/wine.csv', header=None)
df = df_pre.sample(frac=1)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.head(100)

In [ ]:
df.info()

In [ ]:
dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

# 모델 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

# 모델 실행
model.fit(X, Y, epochs=200, batch_size=200)

# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

# 각 에폭마다 모델과 모델의 정확도를 함께 저장
* 모델이 저장될 폴더 지정
* 케라스의 콜백 사용
    - 훈련을 구성하는 단계에서 적용(자동으로 실행)되는 함수의 집합
    - 훈련 도중 모델의 내부 상태나 통계정보 등을 얻기 위해 사용
    ```
    from keras.callbacks import ModelCheckpoint
    ```
        - 에폭마다 모델을 저장하는 콜백

# 와인 종류 예측: 모델 업데이트
### 매 에폭마다 모델과 모델의 정확도를 저장

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import os
import tensorflow as tf

# seed 값 설정
# seed = 0
# np.random.seed(seed)
# tf.set_random_seed(seed)

df_pre = pd.read_csv('./dataset/wine.csv', header=None)
df = df_pre.sample(frac=1)

dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

In [ ]:
# 모델의 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy']
             )

### 모델 저장 폴더 설정

In [ ]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

### ModelCheckpoint 콜백 객체 생성

   - 저장 경로
   - 모니터 대상
       * 훈련셋 정확도: acc
       * 테스트셋 정확도: val_acc
       * 훈련셋 오차: loss
   - 이전 모델보다 나아졌을 때만 저장
     ModelCheckpoint 객체의 save_best_only 파라미터 = True
   - 매 epoch 끝날 때마다 호출되어 모델을 저장함

In [ ]:
# 모델 저장 조건 설정
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 모델 실행 및 저장
model.fit(X, Y, validation_split=0.2, epochs=200, batch_size=200, verbose=0, callbacks=[checkpointer])

# 모델의 업데이트를 그래프로 표현

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

df_pre = pd.read_csv('./dataset/wine.csv', header=None)
df = df_pre.sample(frac=0.15)

dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

# 모델의 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건 설정
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

### fit 메소드는 모델의 훈련과정의 측정값들을 담고 있는 History 객체 반환
* History.history 사전

In [ ]:
history = model.fit(X, Y, validation_split=0.33, epochs=3500, batch_size=500)

In [ ]:
# y_vloss에 테스트셋으로 실험 결과의 오차 값을 저장
y_vloss=history.history['val_loss']

# y_acc 에 훈련셋으로 측정한 정확도의 값을 저장
y_acc=history.history['acc']

# x값을 지정하고 정확도를 파란색으로, 오차를 빨간색으로 표시
x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vloss, "o", c="red", markersize=1)
plt.plot(x_len, y_acc, "o", c="blue", markersize=1)

plt.show()

In [ ]:
history.history

In [ ]:
type(history.history)

In [ ]:
history.history.keys()

In [ ]:
type(history.history['val_loss'])

In [ ]:
len(history.history['val_loss'])

# 학습의 자동 중단
### 훈련이 진행될수록 훈련셋의 정확도는 향상되지만 과적합으로 인해 테스트셋의 결과는 점점 나빠짐
* 학습이 진행되어도 테스트셋 오차가 줄지 않으면 학습을 멈추게 하는 콜백

    ```
    from keras.callbacks import EarlyStopping
    ```
        - 모니터할 값
        - 테스트 오차가 좋지 않아도 몇 번까지 기다릴지 결정

# 와인 종류 예측: 학습의 자동중단 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
import tensorflow as tf

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

df_pre = pd.read_csv('./dataset/wine.csv', header=None)
df = df_pre.sample(frac=0.15)

dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

### 자동 중단을 지원하는 콜백

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [ ]:
# 모델 실행
model.fit(X, Y, validation_split=0.2, epochs=2000, batch_size=500, callbacks=[early_stopping_callback])

# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

### 2000 에폭으로 설정하였으나 777 에폭에서 학습 종료

# 와인 종류 예측: 모델 업데이트와 학습의 자동중단 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

import pandas as pd
import numpy as np
import os
import tensorflow as tf

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

df_pre = pd.read_csv('./dataset/wine.csv', header=None)
df = df_pre.sample(frac=0.15)

dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

model = Sequential()
model.add(Dense(30,  input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

### fit 메소드의 callbacks 파라미터에 콜백 객체를 리스트로 묶어서 지정

In [ ]:
model.fit(X, Y, validation_split=0.2, epochs=3500, batch_size=500, verbose=0, callbacks=[early_stopping_callback,checkpointer])

In [ ]:
model.history.history['val_acc'][-1]

In [ ]:
model.history.history['acc'][-10:]